In [ ]:
import requests
import time
from bs4 import BeautifulSoup

def submit_blast(query_seq, subject_seq):
    url = 'https://blast.ncbi.nlm.nih.gov/Blast.cgi'
    params = {
        'PROGRAM': 'blastp',
        'PAGE_TYPE': 'BlastSearch',
        # 'BLAST_SPEC': 'blast2seq',
        'LINK_LOC': 'blasthome'
    }
    data = {
        'QUERY': query_seq,
        'SUBJECTS': subject_seq,
        'CMD': 'Put'
    }
    response = requests.post(url, params=params, data=data)
    if response.status_code == 200:
        print(response.text) 
        # Parse the response to find the RID
        soup = BeautifulSoup(response.text, 'html.parser')
        rid_input = soup.find('input', {'name': 'RID'})
        if rid_input and rid_input['value']:
            return rid_input['value']
        else:
            raise Exception("RID not found in response")
    else:
        raise Exception(f"Failed to submit BLAST job: {response.status_code}")

def get_blast_results(rid):
    url = 'https://blast.ncbi.nlm.nih.gov/Blast.cgi'
    params = {
        'CMD': 'Get',
        'RID': rid,
        'FORMAT_TYPE': 'Text'
    }
    while True:
        response = requests.get(url, params=params)
        if response.status_code == 200:
            if 'Status=WAITING' in response.text:
                print("Waiting for results...")
                time.sleep(10)
            else:
                alignment_output = re.search(r'Seqdump\.txt:\s+([\s\S]+?)\s+BLASTP 2', response.text)
                if alignment_output:
                    return alignment_output.group(1).strip()
                else:
                    raise Exception("Failed to find alignment output in results")
        else:
            raise Exception(f"Failed to retrieve BLAST results: {response.status_code}")

# Example usage
query_seq = """>Query
UUUUUUUUUUUUUUUUUUUUUPHLSEQLCFFVQAREIADFYEKYALSTQKFINTEELVSTLDTILRKYSPLESSFQLEVGVLSHLLKAQAQISEWKFLPSLVTLHNAHTKLQSWGQTFEKQRPPHLFLWLKLKTLLAKFSFYFHEALSRQTTASEKALTAKANP"""
subject_seq = """>Subject
MGESIPLAAPVPVEQAVLETFFSHLGIFSYDKAKDNVEKEREANKSAGGSWLSLLAALAHLAAAEKVYHSLTYLGQKLGGQSFFSRKDSIRTIYTSLHNELKKVVAGRGAPGGTAPHVEELLPHLSEQLCFFVQARMEIADFYEKMYALSTQKFINTEELVSTLDTILRKYSSRFHHPILSPLESSFQLEVGVLSHLLKAQAQISEWKFLPSLVTLHNAHTKLQSWGQTFEKQRETKKHLFGGQSQKAVQPPHLFLWLMKLKTMLLAKFSFYFHEALSRQTTASEMKALTAKANPDLFGKISSFIRKYDAANVSLIFDNRGSESFQGHGYHHPHSYREAPKGVDQYPAVVSLPSDRPVMHWPNVIMIMTDRASDLNSLEKVVHFYDDKVQSTYFLTRPEPHFTIVVIFESKKSERDSHFISFLNELSLALKNPKVFASLKPGSKG"""

try:
    rid = submit_blast(query_seq, subject_seq)
    print(f"Submitted BLAST job with RID: {rid}")
    alignment_result = get_blast_results(rid)
    print("Alignment Output:")
    print(alignment_result)
except Exception as e:
    print(f"An error occurred: {e}")


In [43]:
# Example usage
query_seq = """>Query
UUUUUUUUUUUUUUUUUUUUUPHLSEQLCFFVQAREIADFYEKYALSTQKFINTEELVSTLDTILRKYSPLESSFQLEVGVLSHLLKAQAQISEWKFLPSLVTLHNAHTKLQSWGQTFEKQRPPHLFLWLKLKTLLAKFSFYFHEALSRQTTASEKALTAKANP"""
subject_seq = """>Subject
MGESIPLAAPVPVEQAVLETFFSHLGIFSYDKAKDNVEKEREANKSAGGSWLSLLAALAHLAAAEKVYHSLTYLGQKLGGQSFFSRKDSIRTIYTSLHNELKKVVAGRGAPGGTAPHVEELLPHLSEQLCFFVQARMEIADFYEKMYALSTQKFINTEELVSTLDTILRKYSSRFHHPILSPLESSFQLEVGVLSHLLKAQAQISEWKFLPSLVTLHNAHTKLQSWGQTFEKQRETKKHLFGGQSQKAVQPPHLFLWLMKLKTMLLAKFSFYFHEALSRQTTASEMKALTAKANPDLFGKISSFIRKYDAANVSLIFDNRGSESFQGHGYHHPHSYREAPKGVDQYPAVVSLPSDRPVMHWPNVIMIMTDRASDLNSLEKVVHFYDDKVQSTYFLTRPEPHFTIVVIFESKKSERDSHFISFLNELSLALKNPKVFASLKPGSKG"""

In [48]:
# Example usage
query_seq = "UUUUUUUUUUUUUUUUUUUUUPHLSEQLCFFVQAREIADFYEKYALSTQKFINTEELVSTLDTILRKYSPLESSFQLEVGVLSHLLKAQAQISEWKFLPSLVTLHNAHTKLQSWGQTFEKQRPPHLFLWLKLKTLLAKFSFYFHEALSRQTTASEKALTAKANP"
subject_seq = "MGESIPLAAPVPVEQAVLETFFSHLGIFSYDKAKDNVEKEREANKSAGGSWLSLLAALAHLAAAEKVYHSLTYLGQKLGGQSFFSRKDSIRTIYTSLHNELKKVVAGRGAPGGTAPHVEELLPHLSEQLCFFVQARMEIADFYEKMYALSTQKFINTEELVSTLDTILRKYSSRFHHPILSPLESSFQLEVGVLSHLLKAQAQISEWKFLPSLVTLHNAHTKLQSWGQTFEKQRETKKHLFGGQSQKAVQPPHLFLWLMKLKTMLLAKFSFYFHEALSRQTTASEMKALTAKANPDLFGKISSFIRKYDAANVSLIFDNRGSESFQGHGYHHPHSYREAPKGVDQYPAVVSLPSDRPVMHWPNVIMIMTDRASDLNSLEKVVHFYDDKVQSTYFLTRPEPHFTIVVIFESKKSERDSHFISFLNELSLALKNPKVFASLKPGSKG"

In [40]:
with open('q_seq.txt', 'w') as file:
        file.write(query_seq)

In [49]:
from Bio.Blast.Applications import NcbiblastnCommandline

cline = NcbiblastnCommandline(cmd='blastp', query=query_seq, subject=subject_seq)
cline

NcbiblastnCommandline(cmd='blastp', query='UUUUUUUUUUUUUUUUUUUUUPHLSEQLCFFVQAREIADFYEKYALSTQKFINTEELVSTLDTILRKYSPLESSFQLEVGVLSHLLKAQAQISEWKFLPSLVTLHNAHTKLQSWGQTFEKQRPPHLFLWLKLKTLLAKFSFYFHEALSRQTTASEKALTAKANP', subject='MGESIPLAAPVPVEQAVLETFFSHLGIFSYDKAKDNVEKEREANKSAGGSWLSLLAALAHLAAAEKVYHSLTYLGQKLGGQSFFSRKDSIRTIYTSLHNELKKVVAGRGAPGGTAPHVEELLPHLSEQLCFFVQARMEIADFYEKMYALSTQKFINTEELVSTLDTILRKYSSRFHHPILSPLESSFQLEVGVLSHLLKAQAQISEWKFLPSLVTLHNAHTKLQSWGQTFEKQRETKKHLFGGQSQKAVQPPHLFLWLMKLKTMLLAKFSFYFHEALSRQTTASEMKALTAKANPDLFGKISSFIRKYDAANVSLIFDNRGSESFQGHGYHHPHSYREAPKGVDQYPAVVSLPSDRPVMHWPNVIMIMTDRASDLNSLEKVVHFYDDKVQSTYFLTRPEPHFTIVVIFESKKSERDSHFISFLNELSLALKNPKVFASLKPGSKG')

In [50]:
print(cline)

blastp -query UUUUUUUUUUUUUUUUUUUUUPHLSEQLCFFVQAREIADFYEKYALSTQKFINTEELVSTLDTILRKYSPLESSFQLEVGVLSHLLKAQAQISEWKFLPSLVTLHNAHTKLQSWGQTFEKQRPPHLFLWLKLKTLLAKFSFYFHEALSRQTTASEKALTAKANP -subject MGESIPLAAPVPVEQAVLETFFSHLGIFSYDKAKDNVEKEREANKSAGGSWLSLLAALAHLAAAEKVYHSLTYLGQKLGGQSFFSRKDSIRTIYTSLHNELKKVVAGRGAPGGTAPHVEELLPHLSEQLCFFVQARMEIADFYEKMYALSTQKFINTEELVSTLDTILRKYSSRFHHPILSPLESSFQLEVGVLSHLLKAQAQISEWKFLPSLVTLHNAHTKLQSWGQTFEKQRETKKHLFGGQSQKAVQPPHLFLWLMKLKTMLLAKFSFYFHEALSRQTTASEMKALTAKANPDLFGKISSFIRKYDAANVSLIFDNRGSESFQGHGYHHPHSYREAPKGVDQYPAVVSLPSDRPVMHWPNVIMIMTDRASDLNSLEKVVHFYDDKVQSTYFLTRPEPHFTIVVIFESKKSERDSHFISFLNELSLALKNPKVFASLKPGSKG


In [51]:
!blastp -query q.txt -subject s.txt -outfmt 5 -out output.txt


In [31]:
from Bio.Blast import NCBIWWW, NCBIXML

def align_sequences(query_sequence, target_sequence):
    # Perform the BLAST alignment
    result_handle = NCBIWWW.qblast("blastp", "blastp", query_sequence, subject=target_sequence)
    
    # Parse the BLAST results
    blast_record = NCBIXML.read(result_handle)
    
    # Extract the alignment information
    alignments = []
    for alignment in blast_record.alignments:
        for hsp in alignment.hsps:
            alignments.append({
                'query': hsp.query,
                'match': hsp.match,
                'subject': hsp.sbjct,
                'score': hsp.score,
                'e_value': hsp.expect
            })
    
    return alignments

In [32]:
# Example usage
query_seq = "YOUR_QUERY_SEQUENCE"
target_seq = "YOUR_TARGET_SEQUENCE"
alignments = align_sequences(query_seq, target_seq)

if alignments:
    for alignment in alignments:
        print("Query: ", alignment['query'])
        print("Match: ", alignment['match'])
        print("Subject: ", alignment['subject'])
        print("Score: ", alignment['score'])
        print("E-value: ", alignment['e_value'])
        print()
else:
    print("No alignments found.")

TypeError: qblast() got an unexpected keyword argument 'subject'

In [26]:
import requests
from bs4 import BeautifulSoup
import time

def query_blast(query_sequence, subject_sequence):
    url = "https://blast.ncbi.nlm.nih.gov/Blast.cgi"
    
    # Parameters specific to your query
    params = {
        "PROGRAM": "blastp",
        "PAGE_TYPE": "BlastSearch",
        "BLAST_SPEC": "blast2seq",
        "LINK_LOC": "blasttab"
    }
    
    # Form data
    data = {
        "QUERY": query_sequence,
        "SUBJECTS": subject_sequence,
        "CMD": "Put"
    }
    
    # Send POST request to submit the job
    response = requests.post(url, params=params, data=data)
    
    # Check if request was successful
    if response.status_code == 200:
        # with open('response.txt', 'w') as file:
        #         file.write(response.content)
            
        # Parse the HTML response to find the RID (Request ID)
        soup = BeautifulSoup(response.content, 'html.parser')
        rid_tag = soup.find('input', {'name': 'RID'})
        if not rid_tag:
            print("Failed to retrieve RID.")
            return None
        else:
            print('RID found')
        rid = rid_tag['value']
        
        # Poll for the results
        while True:
            time.sleep(10)  # Wait for 10 seconds before checking the status
            result_params = {
                "RID": rid,
                "CMD": "Get",
                "FORMAT_TYPE": "Text"
            }
            result_response = requests.get(url, params=result_params)
            if "Status=WAITING" not in result_response.text:
                break
        if result_response:
            print(result_response.text)

        else:
            print('Here')

        
        # Parse the final result page to find the aligned sequences
        result_soup = BeautifulSoup(result_response.content, 'html.parser')
        alignments_section = result_soup.find('pre', {'class': 'seqdump'})
        if alignments_section:
            aligned_sequences = alignments_section.text.strip()
            return aligned_sequences
        else:
            print("Failed to retrieve aligned sequences.")
            return None
    else:
        print(f"Request failed with status code {response.status_code}")
        return None


In [27]:
# Example usage
aligned_sequences = query_blast(query_seq, subject_seq)
if aligned_sequences:
    print("Aligned Sequences:")
    print(aligned_sequences)
else:
    print("Failed to retrieve aligned sequences.")


RID found
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="en" lang="en">
<head>
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
<meta name="jig" content="ncbitoggler"/>
<meta name="ncbitoggler" content="animation:'none'"/>
<title>NCBI Blast:</title>
<script type="text/javascript" src="/core/jig/1.15.2/js/jig.min.js             "></script>
<script type="text/javascript">    jQuery.getScript("/core/alerts/alerts.js", function() {
        galert(['div#header', 'body > *:nth-child(1)'])
    });</script>
<meta http-equiv="Pragma" content="no-cache">
<link rel="stylesheet" type="text/css" href="css/uswds.min.css" media="screen" />
<link rel="stylesheet"  type="text/css" href="https://www.ncbi.nlm.nih.gov/style-guide/static/nwds/css/nwds.css"/>

<link rel="stylesheet" href="css/headerNew.css?v=1"/>
<link rel="stylesheet" href="https://use.fontawe